In [25]:
#background subtraction
import cv2
import imutils
import numpy as np
import math
import time
import pyautogui

bg = None #None object means empty

In [26]:

def run_avg(image, aWeight):
    
    global bg    

    if bg is None:
        bg = image.copy().astype("float")
        return
    
    cv2.accumulateWeighted(image, bg, aWeight)

In [27]:

def segment(image, threshold=10):
    global bg
  
    diff = cv2.absdiff(bg.astype("uint8"), image)

    thresholded = cv2.threshold(diff,
                                threshold,
                                255,
                                cv2.THRESH_BINARY)[1] 
    
    (_, cnts, _) = cv2.findContours(thresholded.copy(),
                                    cv2.RETR_EXTERNAL,
                                    cv2.CHAIN_APPROX_SIMPLE)

    if len(cnts) == 0:
        return
    else:
 
        segmented = max(cnts, key=cv2.contourArea)
        return (thresholded, segmented)

In [28]:

if __name__ == "__main__":

    aWeight = 0.5

    camera = cv2.VideoCapture(0)

    top, right, bottom, left = 30, 420, 245, 660

    num_frames = 0

    i=0
    latest20=[]
    while(True):
      
        (grabbed, frame) = camera.read()   

        frame = imutils.resize(frame, width=700)

        frame = cv2.flip(frame, 1)

        clone = frame.copy()

        (height, width) = frame.shape[:2]

        roi = frame[top:bottom, right:left]

        gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        
        gray = cv2.GaussianBlur(gray, (7, 7), 0)
       
        if num_frames < 30:
            run_avg(gray, aWeight)
        else:

            hand = segment(gray)

            if hand is not None:
       
                (thresholded, segmented) = hand

                cv2.drawContours(clone, [segmented + (right, top)], -1, (0, 0, 255))
                
                drawing = np.zeros(roi.shape,np.uint8)
            
                hull = cv2.convexHull(segmented)
                
                #define area of hull and area of hand
                areahull = cv2.contourArea(hull)
                areacnt = cv2.contourArea(segmented)
                if(areacnt!=0):
                #find the percentage of area not covered by hand in convex hull
                    arearatio=((areahull-areacnt)/areacnt)*100
            
                    if(arearatio>12):
                        pyautogui.keyDown('space')
                        pyautogui.keyUp('space') 
                    
                
                cv2.drawContours(drawing,[segmented],-1,(0,255,0),0)
                cv2.drawContours(drawing,[hull],-1,(0,0,255),0)
        
                cv2.imshow("drawing",drawing)
                cv2.imshow("Thesholded", thresholded)
                
                
               
        cv2.rectangle(clone, (left, top), (right, bottom), (0,255,0), 2)

        num_frames += 1

        cv2.imshow("Video Feed", clone)

        keypress = cv2.waitKey(1) & 0xFF

        if keypress == ord("q"):
            break

camera.release()
cv2.destroyAllWindows()